In [3]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn scikit-learn

# Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import make_pipeline
import google.colab.drive as drive

In [4]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define paths to your folders
macro_folder = '/content/drive/MyDrive/macro_folder'
financial_folder = '/content/drive/MyDrive/financial_folder'

def load_data(stock_path, macro_path):
    """
    Load stock and macroeconomic data from CSV files.

    Parameters:
    stock_path (str): Path to the stock CSV file
    macro_path (str): Path to the macroeconomic CSV file

    Returns:
    tuple: (stock_df, macro_df) containing the loaded dataframes
    """
    # Load stock data
    stock = pd.read_csv(stock_path)
    stock['Date'] = pd.to_datetime(stock['Date'])
    stock = stock.set_index('Date')

    # Load macro data
    macro = pd.read_csv(macro_path)
    macro['date'] = pd.to_datetime(macro['date'])
    macro = macro.set_index('date')

    return stock, macro

# Define file paths for each country
countries = ['brazil', 'india', 'south_africa']
stock_paths = {country: f'{financial_folder}/{country}_stock.csv' for country in countries}
macro_paths = {country: f'{macro_folder}/{country}_macro.csv' for country in countries}

# Load data for each country
country_data = {}
for country in countries:
    print(f"Processing {country}...")
    try:
        stock_df, macro_df = load_data(stock_paths[country], macro_paths[country])
        country_data[country] = {'stock': stock_df, 'macro': macro_df}
        print(f"Successfully loaded {country} data")
    except Exception as e:
        print(f"Error loading {country} data: {e}")

# Display data samples
for country in country_data:
    print(f"\n{country.title()} stock data shape: {country_data[country]['stock'].shape}")
    print(country_data[country]['stock'].head())
    print(f"\n{country.title()} macro data shape: {country_data[country]['macro'].shape}")
    print(country_data[country]['macro'].head(10))


Mounted at /content/drive
Processing brazil...


<ipython-input-4-fad3373b7b9c>:21: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  stock['Date'] = pd.to_datetime(stock['Date'])


Successfully loaded brazil data
Processing india...
Successfully loaded india data
Processing south_africa...
Successfully loaded south_africa data

Brazil stock data shape: (121, 1)
                             close
Date                              
2015-04-30 00:00:00-03:00  56229.0
2015-05-31 00:00:00-03:00  52760.0
2015-06-30 00:00:00-03:00  53081.0
2015-07-31 00:00:00-03:00  50865.0
2015-08-31 00:00:00-03:00  46626.0

Brazil macro data shape: (29, 5)
                     GDP         CPI  UNEMPLOYMENT       IMPORTS       EXPORTS
date                                                                          
2016-03-31  1.795693e+12  150.482594        11.580  2.166864e+11  2.238633e+11
2016-06-30  1.795693e+12  150.482594        11.580  2.166864e+11  2.238633e+11
2016-09-30  1.795693e+12  150.482594        11.580  2.166864e+11  2.238633e+11
2016-12-31  1.795693e+12  150.482594        11.580  2.166864e+11  2.238633e+11
2017-03-31  2.063515e+12  155.668786        12.792  2.435106e+11

In [10]:
def create_regime_labels(df, window=6, threshold=0.1):
    """
    Classify market regimes:
    0 = Bearish, 1 = Range-bound, 2 = Bullish

    Parameters:
    df : DataFrame with financial and macro data
    window : lookback period for rolling calculations
    threshold : threshold for determining regime boundaries

    Returns:
    DataFrame with added 'regime' column
    """
    # Create a copy to avoid modifying the original
    result = df.copy()

    # Calculate GDP_change if it doesn't exist but GDP does
    if 'GDP_change' not in result.columns and 'GDP' in result.columns:
        result['GDP_change'] = result['GDP'].pct_change()

    # Calculate return_1m if it doesn't exist but close does
    if 'return_1m' not in result.columns and 'close' in result.columns:
        result['return_1m'] = result['close'].pct_change()

    # Calculate other macro changes if needed
    for col in ['CPI', 'UNEMPLOYMENT', 'IMPORTS', 'EXPORTS']:
        if f'{col}_change' not in result.columns and col in result.columns:
            result[f'{col}_change'] = result[col].pct_change()

    # Calculate rolling mean and standard deviation of close prices
    rolling_mean = result['close'].rolling(window=window).mean()
    rolling_std = result['close'].rolling(window=window).std()

    # Create upper and lower bounds
    upper_bound = rolling_mean + (rolling_std * threshold)
    lower_bound = rolling_mean - (rolling_std * threshold)

    # Initialize regime column as range-bound (1)
    result['regime'] = 1

    # Classify bullish regime (2) - price above upper bound
    result.loc[result['close'] > upper_bound, 'regime'] = 2

    # Classify bearish regime (0) - price below lower bound
    result.loc[result['close'] < lower_bound, 'regime'] = 0

    # Use macro indicators if available
    if 'GDP_change' in result.columns and 'return_1m' in result.columns:
        # For locations where we have valid macro data:
        valid_mask = result['GDP_change'].notna() & result['return_1m'].notna()

        # Strong bullish conditions: positive GDP growth and positive returns
        strong_bullish = valid_mask & (result['GDP_change'] > 0) & (result['return_1m'] > 0)
        result.loc[strong_bullish, 'regime'] = 2

        # Strong bearish conditions: negative GDP growth and negative returns
        strong_bearish = valid_mask & (result['GDP_change'] < 0) & (result['return_1m'] < 0)
        result.loc[strong_bearish, 'regime'] = 0

    return result



In [13]:
from sklearn.impute import SimpleImputer

# Add imputation to your pipeline
model = make_pipeline(
    SimpleImputer(strategy='mean'),  # Impute missing values
    StandardScaler(),
    SVC(class_weight='balanced', probability=True)
)

In [7]:
from sklearn.impute import SimpleImputer

# Add imputation to your pipeline
model = make_pipeline(
    SimpleImputer(strategy='mean'),  # Impute missing values
    StandardScaler(),
    SVC(class_weight='balanced', probability=True)
)

In [12]:
def train_regime_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Create pipeline with scaling and SVC
    model = make_pipeline(
        StandardScaler(),
        SVC(class_weight='balanced', probability=True)
    )

    # Hyperparameter grid
    param_grid = {
        'svc__C': [0.1, 1, 10],
        'svc__gamma': ['scale', 'auto'],
        'svc__kernel': ['rbf', 'poly']
    }

    # Time-series cross-validation
    tscv = TimeSeriesSplit(n_splits=3)
    grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred))

    return best_model

# Train regime models
regime_models = {}
# Define features
features = ['return_1m', 'return_3m', 'volatility_3m', 'GDP_change', 'CPI_change', 'UNEMPLOYMENT_change', 'IMPORTS_change', 'EXPORTS_change']

# Create and populate the 'data' dictionary
data = {}
for country, data_dict in country_data.items():
    try:
        # Get stock and macro DataFrames
        stock_df = data_dict['stock'].copy()
        macro_df = data_dict['macro'].copy()

        # Make both indices timezone-naive by removing timezone info
        if isinstance(stock_df.index, pd.DatetimeIndex) and stock_df.index.tz is not None:
            stock_df.index = stock_df.index.tz_localize(None)

        if isinstance(macro_df.index, pd.DatetimeIndex) and macro_df.index.tz is not None:
            macro_df.index = macro_df.index.tz_localize(None)

        # Merge the DataFrames
        merged_df = pd.merge(stock_df, macro_df, left_index=True, right_index=True, how='inner')

        # Apply create_regime_labels to add 'regime' column
        data[country] = create_regime_labels(merged_df)

        print(f"Successfully processed {country} data: {data[country].shape[0]} rows")
    except Exception as e:
        print(f"Error processing {country}: {str(e)}")

# Ensure all required features exist before training
for country in countries:
    if country not in data:
        print(f"Skipping {country} (no data available)")
        continue

    df = data[country]
    print(f"\nPreparing to train model for {country.title()}:")

    # Check for missing features and calculate if possible
    missing_features = []
    for feature in features:
        if feature not in df.columns:
            missing_features.append(feature)

            # Try to calculate missing features
            if feature == 'return_3m' and 'close' in df.columns:
                df['return_3m'] = df['close'].pct_change(3)
            elif feature == 'volatility_3m' and 'return_1m' in df.columns:
                df['volatility_3m'] = df['return_1m'].rolling(3).std()
            elif feature.endswith('_change') and feature[:-7] in df.columns:
                base_col = feature[:-7]  # Extract base column name (e.g., 'GDP' from 'GDP_change')
                df[feature] = df[base_col].pct_change()

    if missing_features:
        print(f"Calculated missing features: {missing_features}")

    # Check which features are now available
    available_features = [f for f in features if f in df.columns]

    if len(available_features) < 3:
        print(f"Not enough features available for {country}, skipping")
        continue

    print(f"Training with features: {available_features}")

    # Handle any remaining NaN values
    X = df[available_features].dropna()
    y = df.loc[X.index, 'regime']

    if len(X) < 20:
        print(f"Not enough valid data points ({len(X)}) for {country} after removing NaNs")
        continue

    print(f"Training on {len(X)} valid samples")
    regime_models[country] = train_regime_model(X, y)


Successfully processed brazil data: 0 rows
Successfully processed india data: 29 rows
Successfully processed south_africa data: 23 rows

Preparing to train model for Brazil:
Calculated missing features: ['return_3m', 'volatility_3m']
Training with features: ['return_1m', 'return_3m', 'volatility_3m', 'GDP_change', 'CPI_change', 'UNEMPLOYMENT_change', 'IMPORTS_change', 'EXPORTS_change']
Not enough valid data points (0) for brazil after removing NaNs

Preparing to train model for India:
Calculated missing features: ['return_3m', 'volatility_3m']
Training with features: ['return_1m', 'return_3m', 'volatility_3m', 'GDP_change', 'CPI_change', 'UNEMPLOYMENT_change', 'IMPORTS_change', 'EXPORTS_change']
Training on 26 valid samples
Best Parameters: {'svc__C': 0.1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Accuracy: 0.00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0
          

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

Best Parameters: {'svc__C': 0.1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Accuracy: 0.50
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       0.00      0.00      0.00         2

    accuracy                           0.50         4
   macro avg       0.25      0.50      0.33         4
weighted avg       0.25      0.50      0.33         4



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
